<a href="https://colab.research.google.com/github/rodmorx/DSP_DT_WS24/blob/main/DSProjekt_DT_WS24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Sience Projekt WS24 - Dream Team

In [1]:
pip install news-please

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.9/95.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.6/524.6 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from newsplease import NewsPlease
import pandas as pd
import urllib.parse
## Can You READ THIS DREAM TEAM

In [3]:
# Function to extract the domain (i.e., the newspaper/source) from the URL
def extract_newspaper(url):
    parsed_url = urllib.parse.urlparse(url)
    domain = parsed_url.netloc
    return domain.replace('www.', '')  # Clean up 'www.' if present

In [4]:
# Function to scrape an article from a URL and return relevant information
def scrape_article(url):
    try:
        article = NewsPlease.from_url(url)
        return {
            'title': article.title,
            'authors': article.authors,
            'date_publish': article.date_publish,
            'maintext': article.maintext,
            'url': url,
            'newspaper': extract_newspaper(url)  # Add newspaper information
        }
    except Exception as e:
        # Return None if there's any error (e.g., 404, timeout)
        print(f"Error scraping {url}: {e}")
        return None

In [5]:
#Function to scrape multicle articles and return a dataframe with relevant information
def scrape_articles(url_list):
    # Initialize an empty list to store scraped article data
    articles_data = []

    # Loop through each URL and scrape the article
    for url in url_list:
        article_data = scrape_article(url)
        if article_data:
            articles_data.append(article_data)

    # Convert the list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(articles_data)

    return df

In [6]:
# Test Usage
urls = [
    "https://www.abendblatt.de/hamburg/hamburg-mitte/article407529183/elbtower-kauft-tuerkische-firma-hamburgs-kuenftiges-wahrzeichen.html",
    "https://www.sueddeutsche.de/politik/israel-hisbollah-libanon-bank-gold-geld-krankenhaus-lux.TxxwzAtVsoRuvgqkHfMiuy",
    "https://www.morgenpost.de/politik/article407533911/gruene-verbieten-hunde-wie-csu-zu-diesem-schwachsinn-kommt.html"
]

# Scrape articles and create a DataFrame
articles_df = scrape_articles(urls)

# Display the dataframe
print(articles_df.head())

                                               title            authors  \
0  Elbtower Hamburg: Drei Kaufinteressenten sind ...   [Ulrich Gaßdorf]   
1   Israels Armee sucht den Goldschatz der Hisbollah    [Bernd Dörries]   
2  Grüne verbieten Hunde? Wie CSU zu diesem „Schw...  [Daniel Weidmann]   

         date_publish                                           maintext  \
0 2024-10-28 13:25:00  Hamburg. Zu den möglichen Investoren soll eine...   
1 2024-10-22 14:56:01  Der Direktor des Krankenhauses, ein Parlaments...   
2 2024-10-24 11:18:36  Berlin. Der CSU-Generalsekretär behauptet, die...   

                                                 url        newspaper  
0  https://www.abendblatt.de/hamburg/hamburg-mitt...    abendblatt.de  
1  https://www.sueddeutsche.de/politik/israel-his...  sueddeutsche.de  
2  https://www.morgenpost.de/politik/article40753...    morgenpost.de  


In [7]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         3 non-null      object        
 1   authors       3 non-null      object        
 2   date_publish  3 non-null      datetime64[ns]
 3   maintext      3 non-null      object        
 4   url           3 non-null      object        
 5   newspaper     3 non-null      object        
dtypes: datetime64[ns](1), object(5)
memory usage: 272.0+ bytes


In [8]:
articles_df.head()

,title,authors,date_publish,maintext,url,newspaper
0,Elbtower Hamburg: Drei Kaufinteressenten sind ...,[Ulrich Gaßdorf],2024-10-28 13:25:00,Hamburg. Zu den möglichen Investoren soll eine...,https://www.abendblatt.de/hamburg/hamburg-mitt...,abendblatt.de
1,Israels Armee sucht den Goldschatz der Hisbollah,[Bernd Dörries],2024-10-22 14:56:01,"Der Direktor des Krankenhauses, ein Parlaments...",https://www.sueddeutsche.de/politik/israel-his...,sueddeutsche.de
2,Grüne verbieten Hunde? Wie CSU zu diesem „Schw...,[Daniel Weidmann],2024-10-24 11:18:36,"Berlin. Der CSU-Generalsekretär behauptet, die...",https://www.morgenpost.de/politik/article40753...,morgenpost.de


In [9]:
print(articles_df.loc[0, 'maintext'])

Hamburg. Zu den möglichen Investoren soll eine bekannte türkische Firma gehören. Doch auch zwei Konsortien aus Hamburg hoffen auf den Zuschlag.


In [10]:
pip install newspaper_scraper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 32.1 MB/s eta 0:00:00


In [11]:
import newspaper_scraper as nps

In [12]:
#with nps.DeBild(db_files='articles_debild.db') as news: #this does not compile, we should find a way to write files while using colab
  #news.index_articles_by_date_range('2023-09-01', '2023-09-30') #fron 1st to 30th september 2023
  #news.scrape_public_articles()
  #it's also possible to apply spacy's nlp function directly with the scraping library
  #news.nlp()

TypeError: DeBild.__init__() got an unexpected keyword argument 'db_files'

In [21]:
import requests
from bs4 import BeautifulSoup
import csv
from urllib.parse import urlparse, urljoin

# List of news websites to scrape
urls_to_scrape = [
    "https://www.spiegel.de",   # Der Spiegel
    "https://www.zeit.de",      # Die Zeit
    "https://www.sueddeutsche.de"  # Süddeutsche Zeitung
]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
}

article_urls = set()  # Use a set to avoid duplicates

# Patterns that likely indicate article URLs
article_patterns = ["/artikel/", "/news/", "/politik/", "/wirtschaft/", "/sport/", "/202", "/2023", "/2024"]

# Exclude URLs with these keywords
exclude_keywords = ["contact", "about", "impressum", "privacy", "datenschutz", "jobs", "login"]

for base_url in urls_to_scrape:
    try:
        response = requests.get(base_url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        parsed_base_url = urlparse(base_url).netloc

        for link in soup.find_all("a", href=True):
            href = link['href']
            full_url = urljoin(base_url, href)  # Convert to absolute URL

            # Ensure the URL belongs to the base domain and matches article patterns
            parsed_url = urlparse(full_url)
            if (parsed_url.netloc == parsed_base_url and
                any(pattern in full_url for pattern in article_patterns) and
                not any(keyword in full_url for keyword in exclude_keywords)):
                article_urls.add(full_url)

        # Stop when we have 1000 unique URLs
        if len(article_urls) >= 1000:
            break

    except requests.RequestException as e:
        print(f"Error fetching {base_url}: {e}")

# Save URLs to a CSV file
with open("filtered_news_article_urls.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    for article_url in list(article_urls)[:1000]:  # Limit to 1000 URLs
        writer.writerow([article_url])

print("Filtered article URLs saved to filtered_news_article_urls.csv")

Filtered article URLs saved to filtered_news_article_urls.csv


In [25]:
urls_file = pd.read_csv("news_article_urls.csv")

In [26]:
urls_file.head(10)

,https://boersen.manager-magazin.de/kursinformation/DE0008469008/
0,https://gutscheine.spiegel.de/alle-shops
1,https://sz-magazin.sueddeutsche.de/gesundheit/...
2,https://shop.zeit.de/geschenke-zon
3,https://streaming-guide.spiegel.de/streamingdi...
4,https://stellenmarkt.sueddeutsche.de/jobs/medi...
5,https://seniorenportal.spiegel.de/
6,https://spiele.spiegel.de/snake/?utm_campaign=...
7,https://quiz.sueddeutsche.de/quiz/50ceff3b49fd...
8,https://twitter.com/sz
9,https://gruppenkonto.spiegel.de/kuendigung.html


In [24]:
urls_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 1 columns):
 #   Column                                                            Non-Null Count  Dtype 
---  ------                                                            --------------  ----- 
 0   https://boersen.manager-magazin.de/kursinformation/DE0008469008/  223 non-null    object
dtypes: object(1)
memory usage: 1.9+ KB
